Haal gebruikers op die films hebben gerate. Er ontstaat een RDD[(userId, movieId, rating)].

In [2]:
userdata = sc.textFile("../../../schooldata/nieuw/ratings_train.csv")

firstline = userdata.first()

data_normalized = userdata.filter(lambda x: x != firstline)

data_normalized = data_normalized.map(lambda x: x.split("::"))

user_mov_rating = data_normalized.map(lambda x:(int(x[0]), int(x[1]), float(x[2]))).cache()

user_mov_rating.take(5)

[(11973, 11, 3.0),
 (11973, 14, 4.0),
 (11973, 32, 2.0),
 (11973, 36, 3.0),
 (11973, 110, 5.0)]

Haal films op uit de csv en map ze in een RDD zodat er een RDD[(movieId, title)] ontstaat.

In [3]:
moviedata = sc.textFile("../../../schooldata/nieuw/movies.csv")

firstline = moviedata.first()
movies_normalized = moviedata.filter(lambda x: x != firstline)\
.map(lambda x: x.split(','))

mov_id_name = movies_normalized.map(lambda x: (int(x[0]), x[1])).cache()

mov_id_name.take(5)

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)')]

Haal test records uit de csv en map ze in een RDD zodat er een [(userId, movieId, rating)] RDD ontstaat.

In [4]:
testdata = sc.textFile("../../../schooldata/nieuw/ratings_test.csv")
header = testdata.first()
test_normalized = testdata.filter(lambda x: x != header)
test_normalized = test_normalized.map(lambda x: x.split('::'))

test_user_mov_rat = test_normalized.map(lambda x: (int(x[0]), int(x[1]), float(x[2]))).cache()

test_user_mov_rat.take(5)


[(1, 3889, 4.0), (3, 2034, 4.0), (4, 489, 4.0), (5, 1136, 5.0), (8, 454, 4.0)]

groupByKey de films voor (movieId, rating) rdd. Maak vervolgens een (movieId, (len(ratings), avg_rating))) rdd van films per film. Round de (rating) float op 1 decimaal (e.g. 4.0 in plaats van 4.031273920...).

In [5]:
movieId_rating = user_mov_rating.map(lambda x: (x[1], x[2])).groupByKey()

movieId_avgRating = movieId_rating\
.map(lambda x: (x[0], (len(x[1]), round(float(sum(i for i in x[1])) / len(x[1]), 1))))

Sorteer de top movies op aantal keer gerate en map ze vervolgens in een nieuwe rdd als (movieId, avg_rating).

In [6]:
top_rated_movies =  movieId_avgRating.sortBy(lambda x: (-x[1][0])).map(lambda x: (x[0], x[1][1]))

Filter de most rated movies zodat de rdd alleen maar films met rating 4.0 of hoger bevat.

In [7]:
top_rated_movies = top_rated_movies.filter(lambda x: x[1] >= 4.0).map(lambda x: (x[0]))

top_rated_movies.take(50)

[296,
 356,
 593,
 318,
 260,
 110,
 2571,
 527,
 457,
 1210,
 50,
 1196,
 2858,
 1198,
 608,
 47,
 858,
 2959,
 2762,
 4993,
 2028,
 5952,
 1136,
 1197,
 7153,
 1291,
 1214,
 541,
 4226,
 1193,
 1704,
 1089,
 1221,
 1200,
 1617,
 1213,
 293,
 924,
 1206,
 111,
 912,
 4973,
 919,
 778,
 1258,
 2329,
 750,
 1208,
 7361,
 1222]

Maak een rdd met (userId, list(mov1, mov2, mov3....) aan met film values die de user al heeft gezien.

In [8]:
user_seenMovs = user_mov_rating.map(lambda x: (x[0], x[1])).groupByKey().map(lambda x: (x[0], list(x[1])))

In [25]:
user_seenMovs.take(50)

[(81923,
  [34,
   39,
   141,
   144,
   342,
   345,
   348,
   357,
   367,
   410,
   412,
   435,
   440,
   468,
   497,
   515,
   539,
   541,
   587,
   590,
   592,
   597,
   1079,
   1080,
   1081]),
 (16388,
  [6,
   10,
   16,
   17,
   19,
   25,
   28,
   34,
   39,
   41,
   44,
   45,
   46,
   47,
   48,
   50,
   54,
   64,
   66,
   94,
   110,
   111,
   132,
   141,
   150,
   153,
   154,
   163,
   165,
   180,
   188,
   196,
   198,
   207,
   229,
   231,
   233,
   235,
   249,
   253,
   256,
   259,
   260,
   272,
   273,
   277,
   278,
   281,
   288,
   292,
   293,
   296,
   303,
   305,
   306,
   307,
   308,
   312,
   315,
   319,
   328,
   330,
   333,
   337,
   339,
   344,
   345,
   353,
   356,
   357,
   361,
   362,
   364,
   367,
   372,
   377,
   380,
   382,
   410,
   412,
   416,
   417,
   420,
   426,
   427,
   431,
   434,
   442,
   454,
   457,
   471,
   474,
   475,
   477,
   480,
   485,
   502,
   508,
   509,
   514,


Map de eerder uitgelezen tests in een nieuwe RDD van [(userId, movieId)] om dit bij joins met RDD's te kunnen gebruiken, zodat we later een structuur krijgen met de movie die de gebruiker aanbevolen hoort te krijgen en de films die hij heeft gezien/niet heeft gezien.

In [9]:
test_user_mov = test_user_mov_rat.map(lambda x: (x[0], x[1]))

Join de test_user_mov met user_seenMovs en map deze vervolgens in een [(userId, nextMov, [list_of_seen_movies])]

In [10]:
# an rdd of (userId, nextMov, [seenMovieIds])
user_nextMov_seenMovs = user_seenMovs.join(test_user_mov).map(lambda x: (x[0], x[1][1], x[1][0]))

user_nextMov_seenMovs.take(50)

[(131081,
  4056,
  [1,
   6,
   11,
   17,
   21,
   25,
   28,
   32,
   34,
   35,
   36,
   50,
   80,
   83,
   84,
   105,
   110,
   111,
   112,
   149,
   150,
   164,
   165,
   194,
   205,
   208,
   210,
   213,
   222,
   230,
   233,
   237,
   260,
   272,
   281,
   296,
   306,
   307,
   308,
   317,
   318,
   334,
   337,
   348,
   356,
   357,
   368,
   373,
   376,
   377,
   380,
   388,
   389,
   412,
   434,
   440,
   448,
   457,
   468,
   474,
   477,
   480,
   492,
   497,
   500,
   506,
   507,
   508,
   509,
   515,
   523,
   527,
   529,
   531,
   535,
   539,
   549,
   551,
   562,
   586,
   588,
   589,
   590,
   592,
   595,
   597,
   608,
   715,
   720,
   745,
   750,
   767,
   800,
   852,
   854,
   858,
   893,
   898,
   899,
   903,
   904,
   905,
   906,
   908,
   912,
   919,
   920,
   922,
   923,
   924,
   927,
   929,
   930,
   931,
   934,
   936,
   949,
   950,
   951,
   953,
   955,
   965,
   969,
   971,
   973,

Collect de top_rated_movies en user_nextMov_seenMovs rdd's naar lists, zodat we deze in de volgende stap kunnen gebruiken als argumenten in de checkMovies functie.

In [11]:
top_movies = top_rated_movies.collect()
user_seen = user_nextMov_seenMovs.collect()

De functie checkMovies gaat per most rated movie in de meegegeven top_movies argument en scant de films die een gebruiker al heeft gezien (user_seen argument) en zolang de most rated movie uit top_movies zich niet in de bekeken films lijst bevindt van een gebruiker, wordt deze in een nieuwe lijst gemapt en deze wordt uiteindelijke returnd als [(userId, not_yet_seen_movie), (userId, not_yet_seen_movie), ....].

In [12]:
def checkMovies(top_movies, user_seen):
    new_dict = []
    for i in range(len(user_seen)):
        
        counter = 0
        for val in top_movies:
            if counter >= 10:
                break
            if val not in user_seen[i][2]:
                new_dict.append((user_seen[i][0], val))
                counter += 1
    
    return new_dict

user_notSeenMov = checkMovies(top_movies, user_seen)

Hier wordt de user_notSeenMov lijst omgezet in een RDD. Vervolgens wordt de RDD per user de film waarvan verwacht wordt dat hij hem zal bekijken en een lijst met aanbevolen films gemapt. De RDD ziet er als volgt uit: 
[(userId, expected_next_movie, [recommended_mov1, recommended_mov2, recommended_mov3.... recommended_mov10]), 
 (userId, expected_next_movie, [recommended_mov1, recommended_mov2, recommended_mov3.... recommended_mov10]),
 ....].

In [13]:
user_notSeenMov_rdd = sc.parallelize(user_notSeenMov)

user_notSeenMov_rdd = user_notSeenMov_rdd.map(lambda x: x)\
.groupByKey().map(lambda x: (x[0], list(x[1])))

user_notSeenMov_rdd = user_notSeenMov_rdd.join(test_user_mov).map(lambda x: (x[0], x[1][1], x[1][0]))

user_notSeenMov_rdd.collect()

[(65540, 308, [296, 356, 318, 260, 2571, 527, 457, 1210, 50, 1198]),
 (79190, 588, [356, 593, 318, 260, 2571, 527, 457, 1210, 50, 1196]),
 (57350, 4105, [457, 1197, 924, 912, 919, 1259, 1225, 2502, 2324, 923]),
 (49160, 2987, [296, 593, 318, 260, 110, 2571, 527, 457, 1210, 50]),
 (76460, 1356, [110, 527, 457, 2858, 608, 47, 2959, 2762, 2028, 1136]),
 (10, 1136, [296, 593, 318, 110, 2571, 457, 50, 2858, 608, 47]),
 (32780, 7440, [2762, 541, 1704, 912, 4973, 778, 1259, 1080, 17, 58559]),
 (106510, 7153, [296, 356, 593, 318, 260, 110, 2571, 527, 457, 1210]),
 (57360, 50, [296, 356, 593, 318, 260, 110, 2571, 527, 457, 1210]),
 (137210, 50160, [356, 260, 110, 457, 1210, 1196, 1198, 2959, 1197, 1291]),
 (24070, 7162, [296, 356, 593, 318, 260, 527, 457, 1210, 50, 1196]),
 (129710, 1120, [296, 356, 593, 318, 110, 2571, 527, 457, 50, 1196]),
 (127390, 105653, [296, 260, 2571, 527, 457, 1210, 1196, 1198, 608, 858]),
 (106520, 3864, [608, 4993, 5952, 1136, 1197, 7153, 4226, 1089, 1213, 293]),
 (4

Uiteindelijk wordt een functie geschreven die test hoe vaak zich een verwachte film (uit de test set) bevindt in de aanbevolen films. Deze worden in de functie opgesomt en gereturned. Uiteindelijk worden de resultaten geprint.

In [14]:
def getOccurrence(user_mov):
    occurred = 0
    
    for i in range(len(user_mov)):
        expected_mov = user_mov[i][1]
        
        for val in user_mov[i][2]:
            if val in expected_mov:
                occurred += 1
    return occurred

occurred = getOccurrence(user_notSeenMov_rdd.collect())

print('total occurred: ' + str(occurred) + '\nin %: '\
      + str(occurred / len(user_notSeenMov_rdd.collect())))

TypeError: argument of type 'int' is not iterable